In [257]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import pickle
from sklearn import linear_model


In [258]:
#========================================================== Helper Functions ========================================================================

def remove_value(data, column_name, string_to_replace, replace_value='', change_dataType=True):
  data[column_name] = data[column_name].str.replace(string_to_replace, replace_value)
  if change_dataType:
    data[column_name] = data[column_name].astype(int)


def fill_nulls(data):
     mode_dict = {'brand': 'ASUS', 'processor_brand': 'Intel', 'processor_name': 'Core i5', 'processor_gnrtn': '11th', 'ram_gb': '8 GB', 'ram_type': 'DDR4', 'ssd': '512 GB', 'hdd': '0 GB', 'os': '64-bit Windows', 'graphic_card_gb': '0 GB', 'weight': 'Casual', 'warranty': '1 year', 'Touchscreen': 'No', 'msoffice': 'No', 'Price': 64990, 'rating': 'Good Rating', 'Number of Ratings': 0, 'Number of Reviews': 0}



     for col, mode in mode_dict.items():
        data[col].fillna(mode, inplace=True)


In [259]:
# Load Model
def load(file_path):
    loaded_model = pickle.load(open(file_path, 'rb'))
    return loaded_model

In [260]:
# Feature Encoder
def feature_encoder(X, cols):
    for c in cols:
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(list(X[c].values))
    return X

In [261]:
# Execute Model
def execute_model(model, X_test, Y_test, model_name):
    Y_test_pred = model.predict(X_test)


    test_mse = mean_squared_error(Y_test, Y_test_pred)
    print(f" Testing MSE: {test_mse}")




    test_accuracy = accuracy_score(Y_test, Y_test_pred)
    print(f" Testing Accuracy: {test_accuracy}")


In [262]:
# Load Data
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

In [263]:
# Preprocess Data
def preprocess_data(data, encoders, one_hot_encoders):
    # filling nulls
    fill_nulls(data)

    # processor_gnrtn
    mode_value = data['processor_gnrtn'].mode()[0]
    data['processor_gnrtn'].replace('Not Available', mode_value, inplace = True)
    remove_value(data, 'processor_gnrtn', 'th')

    # ram_gb
    remove_value(data,'ram_gb' ,' GB')

    # ssd
    remove_value(data,'ssd' ,' GB')

    # hdd
    remove_value(data,'hdd' ,' GB')

    # os
    data[['os_bits', 'os_type']] = data['os'].str.split(' ', expand=True)
    data = data.drop('os', axis = 1)
    remove_value(data,'os_bits' ,'-bit')

    # graphic_card_gb
    remove_value(data,'graphic_card_gb' ,' GB')

    # warranty
    remove_value(data,'warranty' ,'No warranty', '0', False)
    remove_value(data,'warranty' ,' years', '', False)
    remove_value(data,'warranty' ,' year')

    # Touchscreen
    remove_value(data,'Touchscreen' ,'No', '0', False)
    remove_value(data,'Touchscreen' ,'Yes', '1')

    # msoffice
    remove_value(data,'msoffice' ,'No', '0', False)
    remove_value(data,'msoffice' ,'Yes', '1')

    # rating
    rating_performance_mapping = {
        'Bad Rating': 0,
        'Good Rating': 1
    }

    data['rating'] = data['rating'].map(rating_performance_mapping)

    # ram_type
    ram_performance_mapping = {
        'LPDDR4X': 5,
        'LPDDR4': 4,
        'DDR4': 3,
        'DDR3': 2,
        'DDR5': 6,
        'LPDDR3': 1
    }

    data['ram_type'] = data['ram_type'].map(ram_performance_mapping)
    # data['ram_type'].value_counts()

    # weight
    weight = {
        'Casual':2,
        'ThinNlight':1,
        'Gaming':3,
    }
    data['weight'] = data['weight'].map(weight)



    # Apply LabelEncoders to data
    for col, encoder in encoders.items():
        data[col] = encoder.transform(data[col])



     # Apply One-Hot Encoders to data
    for col, one_hot_encoder in one_hot_encoders.items():
        data = data.drop(col, axis=1)
        data = pd.concat([data, one_hot_encoder], axis=1)


    # Replace target column
    rating_col = data['rating']
    data = data.drop('rating', axis = 1)
    data = pd.concat([data, rating_col], axis=1)

    data = handle_missing_values(data)


    return data

In [264]:
# Handle Missing Values
def handle_missing_values(data):
    # columns_to_exclude = ['ram_type', 'Touchscreen', 'os_bits', 'AMD', 'M1', 'DOS', 'Mac', 'Windows', 'rating','Price']
    # data_filtered = data.drop(columns=columns_to_exclude)


    lower_bound = {'brand': -5.0, 'processor_name': -2.5, 'processor_gnrtn': 8.5, 'ram_gb': -2.0, 'ram_type': 3.0, 'ssd': -128.0, 'hdd': -768.0, 'graphic_card_gb': -3.0, 'weight': -0.5, 'warranty': -1.5, 'Touchscreen': 0.0, 'msoffice': -1.5, 'Price': -19869.0, 'Number of Ratings': -231.0, 'Number of Reviews': -29.25, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': -1.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}
    upper_bound = {'brand': 11.0, 'processor_name': 9.5, 'processor_gnrtn': 12.5, 'ram_gb': 14.0, 'ram_type': 3.0, 'ssd': 896.0, 'hdd': 1280.0, 'graphic_card_gb': 5.0, 'weight': 3.5, 'warranty': 2.5, 'Touchscreen': 0.0, 'msoffice': 2.5, 'Price': 155675.0, 'Number of Ratings': 385.0, 'Number of Reviews': 48.75, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': 2.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}

    s = ['ram_type' ,'Touchscreen', 'os_bits' , 'AMD' , 'M1' , 'DOS' , 'Mac' , 'Windows' ]

    # Replace the outliers with lower_bound or upper_bound values
    for col in data.iloc[:,:-1]:
        if col not in s:
            data.loc[data[col] < lower_bound[col], col] = lower_bound[col]
            data.loc[data[col] > upper_bound[col], col] = upper_bound[col]

    # sns.boxplot(data=data,palette='rainbow',orient='h')
    # data_filtered = data.drop(columns=columns_to_exclude)
    return data

In [265]:
    # Load Model
    decision_tree_model = load('ms2 model classification.sav')

    # Load saved LabelEncoders
    encoders = load('ms2 encoders classification.sav')
    one_hot_encoders = load('ms2 one hot encoders classification.sav')

    # Load Test Data
    test_data = load_data('ElecDeviceRatingPrediction_Milestone2.csv')


    # Preprocess Test Data
    test_data = preprocess_data(test_data, encoders, one_hot_encoders)

    # Ensure test data has the same features as training data
    X_test = test_data.iloc[:, :-1]

    # Execute Model
    Y_test = test_data['rating']

    execute_model(decision_tree_model, X_test, Y_test, "Decision Tree")


 Testing MSE: 0.07883817427385892
 Testing Accuracy: 0.921161825726141
